In [1]:
import pandas as pd
import numpy as np
import re
import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.corpus import stopwords
import enchant
from enchant.checker import SpellChecker
import enchant

In [2]:
import sys
print("Python version")
print (sys.version)

Python version
3.7.8 (tags/v3.7.8:4b47a5b6ba, Jun 28 2020, 08:53:46) [MSC v.1916 64 bit (AMD64)]


In [4]:
df = pd.read_csv('../datasets/raw data/raw_data.csv')

# Making the text more uniform

Remove urls

In [5]:
df['preproc'] = df.body.replace(r'http\S+', '', regex=True)

Change acronyms to words for uniformity. 
* https://www.netlingo.com/acronyms.php
* https://blog.adioma.com/internet-acronyms-intro-list-infographic/

In [12]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from util.acronyms_smileys import acronyms

In [13]:
df.preproc = df.preproc.str.lower()
# the ’ was causing issues, took a while to notice
df.preproc = df.preproc.str.replace('’', '\'')

In [14]:
df.preproc = df.preproc.apply(lambda x: ' '.join(acronyms.get(word, word) for word in x.split()))

List of emojis defined from https://en.wikipedia.org/wiki/List_of_emoticons , https://emojipedia.org/people/

Other references:
* https://www.urbandictionary.com/define.php?term=%F0%9F%92%80
* https://www.urbandictionary.com/define.php?term=%F0%9F%94%A5

In [8]:
import emoji
# list of tagged emoticons from above links
from acronyms_smileys import smileys
from acronyms_smileys import sent_acronyms
# for removing untagged emoji
import demoji
#demoji.download_codes()

In [9]:
def replace_repeating_emoji(text):
    uniques = set()
    final_string = list()
    text_arr = [item for item in emoji.get_emoji_regexp().split(text) if not item == '']
    for e in text_arr:
        # for some reason even though it is defined as '❤', when its 
        # imported, it gets loaded as '❤❤'
        if e == '❤':
            e = '❤❤'
        if not bool(emoji.get_emoji_regexp().search(e)):
            final_string.append(smileys.get(e, e))
        else:
            if e not in uniques:
                uniques.add(e)
                final_string.append(smileys.get(e, e))
    return ' '.join(final_string)

In [10]:
# find repeating emoticons and remove repetitions, and tag emoticons
df.preproc = df.preproc.apply(lambda x: replace_repeating_emoji(x) if (bool(emoji.get_emoji_regexp().search(x)) and bool(re.search(r'(.)\1', x))) else x)

In [11]:
# remove untagged emoticons
df.preproc = df.preproc.apply(lambda x : demoji.replace(x, ''))

Set sentiment on acronyms (such as 'lol')

In [12]:
df.preproc = df.preproc.apply(lambda x: ' '.join(sent_acronyms.get(word, word) for word in x.split()))

Remove hashtags

In [13]:
df.preproc = df.preproc.apply(lambda x: ' '.join([word for word in x.split() if '#' not in word]))

In [14]:
df.loc[df.preproc.str.contains('#')]

,body,positive,negative,neutral,rated,comment_id,video_id,date,preproc


Replace negations with "not"

In [15]:
negations = ['don\'t', 'aint' 'aren\'t', 'couldn\'t','didn\'t', 
             'doesn\'t', 'hadn\'t', 'hasn\'t', 'haven\'t', 'isn\'t', 
             'mightn\'t', 'mustn\'t', 'needn\'t', 'shouldn\'t', 'wasn\'t', 
             'weren\'t', 'won\'t', 'wouldn\'t', 'nor', 'not', 'cant', 'dont',
            'arent', 'couldnt', 'didnt', 'doesnt', 'hadnt', 'hasnt', 'havent',
            'isnt', 'mightnt', 'mustnt', 'neednt', 'shouldnt', 'wasnt',
            'werent', 'wont', 'wouldnt']
regx = r'\b(?:{})\b'.format('|'.join(negations))
df.preproc = df.preproc.str.replace(regx, 'not')

In [16]:
# Remove negations from stop list, add two missing contractions
stopwords_list = stopwords.words('english')
stopwords_list = [el for el in stopwords_list if el not in negations]
missing_words = ['i\'m', 'i\'d']
stopwords_list.extend(missing_words)

10 most commonly used words

In [17]:
pd.Series(' '.join(df.preproc).lower().split()).value_counts()[:10]

the     4158
i       3446
this    3072
you     2635
is      2582
a       2337
to      2195
and     1819
not     1598
of      1454
dtype: int64

In [18]:
def get_common_stopwords(stop_words, n=5):
    most_freq_words = pd.Series(' '.join(df.preproc).lower().split()).value_counts()[:int(n*2)].keys().to_numpy()
    common_stopwords = [i for i in most_freq_words if i in stop_words]
    return common_stopwords[0:n]

Remove most common stop words

In [19]:
common_stopwords = get_common_stopwords(stopwords_list, n=35)
df.preproc = df.preproc.apply(lambda x: ' '.join([word for word in x.split() if word not in (common_stopwords)]))

Remove articles (the, an/a)

In [20]:
articles = ['the', 'an', 'a']
df.preproc = df.preproc.apply(lambda x: ' '.join([word for word in x.split() if word not in (articles)]))

In [21]:
df.head(5)

,body,positive,negative,neutral,rated,comment_id,video_id,date,preproc
0,*stretched past the 10 minute mark for ad reve...,0,1,0,1,UgztP4lVR-Epv5HlSXN4AaABAg,ItYOdWRo0JY,2020-01-10T20:24:33Z,"*stretched past 10 minute mark ad revenue, see!*"
1,That makes no sense you sold a new phone for n...,0,1,0,1,Ugw3sxLwikkGsUza7hh4AaABAg,ItYOdWRo0JY,2020-03-22T19:13:19Z,makes sense sold new phone nothing ?
2,WHY IS HE SO HAPPY HE JUST LOST $900 DOLLARS,0,0,1,1,Ugxe5-sz9eHyFCvZX4l4AaABAg,ItYOdWRo0JY,2020-10-30T12:27:57Z,happy lost $900 dollars
3,Does it work on iPads,0,0,1,1,UgxInsxWP8cnbFEh1Cp4AaABAg,ItYOdWRo0JY,2020-07-21T16:38:40Z,does work ipads
4,Yo ass better not have donated.. that cord mig...,0,0,1,1,Ugwn6_K1_792ARqfY4h4AaABAg,ItYOdWRo0JY,2019-12-23T13:35:04Z,yo ass better not donated.. cord might been ba...


Remove punctuation

In [22]:
import string
df.preproc = df.preproc.str.replace('[{}]'.format(string.punctuation), '')

In [23]:
df.head(5)

,body,positive,negative,neutral,rated,comment_id,video_id,date,preproc
0,*stretched past the 10 minute mark for ad reve...,0,1,0,1,UgztP4lVR-Epv5HlSXN4AaABAg,ItYOdWRo0JY,2020-01-10T20:24:33Z,stretched past 10 minute mark ad revenue see
1,That makes no sense you sold a new phone for n...,0,1,0,1,Ugw3sxLwikkGsUza7hh4AaABAg,ItYOdWRo0JY,2020-03-22T19:13:19Z,makes sense sold new phone nothing
2,WHY IS HE SO HAPPY HE JUST LOST $900 DOLLARS,0,0,1,1,Ugxe5-sz9eHyFCvZX4l4AaABAg,ItYOdWRo0JY,2020-10-30T12:27:57Z,happy lost 900 dollars
3,Does it work on iPads,0,0,1,1,UgxInsxWP8cnbFEh1Cp4AaABAg,ItYOdWRo0JY,2020-07-21T16:38:40Z,does work ipads
4,Yo ass better not have donated.. that cord mig...,0,0,1,1,Ugwn6_K1_792ARqfY4h4AaABAg,ItYOdWRo0JY,2019-12-23T13:35:04Z,yo ass better not donated cord might been bad ...


Remove repeating vowels and consonants

In [24]:
# https://stackoverflow.com/questions/46701245/how-to-replace-multiple-consecutive-repeating-characters-into-1-character-in-pyt
df.preproc = df.preproc.apply(lambda x: ' '.join([re.sub(r'[^\w\s]|(.)(?=\1)', '', word) for word in x.split()]))

Tag any sequence of "ha" or "ah" (for example, "ahaha" or "haha") as a "laugh"

In [25]:
df.preproc = df.preproc.apply(lambda x: ' '.join([re.sub(r'([ha]+[ah]+).*\1', r'laugh', word) for word in x.split()]))

Remove numbers

In [26]:
df.preproc = df.preproc.str.replace('\d+', '')

In [27]:
def assign_rating(row):
    if row['positive'] == 1:
        return 1
    elif row['negative'] == 1:
        return 0
    else:
        return -1

In [28]:
# 0 - negative, 1 - positive, -1 neutral
df['rating'] = df.apply(lambda row: assign_rating(row), axis=1)

Write preprocessed column and the rating to a file

In [29]:
# copy relevant columns for later to file write
data_no_trans_stem = df.filter(['comment_id', 'preproc', 'rating'], axis=1)
data_no_trans_stem.columns = ['comment_id', 'body', 'rating']
#data_trans = df.filter(['comment_id', 'preproc', 'rating'], axis=1)
#data_trans.columns = ['comment_id', 'body', 'rating']
data_stem = df.filter(['comment_id', 'preproc', 'rating'], axis=1)
data_stem.columns = ['comment_id', 'body', 'rating']
#data_trans_stem = df.filter(['comment_id', 'preproc', 'rating'], axis=1)
#data_trans_stem.columns = ['comment_id', 'body', 'rating']
data_long_cmnts = df.filter(['comment_id', 'preproc', 'rating'], axis=1)
data_long_cmnts.columns = ['comment_id', 'body', 'rating']

Fix as many spelling errors as possible

In [30]:
#spell_checker = SpellChecker("en_UK","en_US")
#def correct_error(body):
#    spell_checker.set_text(body)
#    for err in spell_checker:
#        if len(err.suggest())>0: 
#            sug = err.suggest()[0]
#            err.replace(sug)
#    return spell_checker.get_text()
#data_trans.body = data_trans.body.apply(lambda row: correct_error(row))
#data_trans_stem.body = data_trans_stem.body.apply(lambda row: correct_error(row))
#data_trans.body = data_trans.body.str.lower()
#data_trans_stem.body = data_trans_stem.body.str.lower()

Stem the words

In [31]:
snows = nltk.stem.SnowballStemmer('english')
data_stem.body = data_stem.body.apply(lambda x: ' '.join([snows.stem(word) for word in x.split()]))
#data_trans_stem.body = data_trans_stem.body.apply(lambda x: ' '.join([snows.stem(word) for word in x.split()]))

Generate data with comments that have more than 5 words after all the preprocessing

In [32]:
snows.stem('why')

'whi'

In [33]:
#data_long_cmnts = data_long_cmnts[data_long_cmnts.body.str.split().str.len() >= 5]
#data_long_cmnts.shape

(6440, 3)

In [34]:
data_no_trans_stem.to_csv('../datasets/preprocessed data/unstemmed/preproc_no_trans_stem.csv', index=False)
#data_trans.to_csv('../data/preproc_trans.csv', index=False)
data_stem.to_csv('../datasets/preprocessed data/stemmed/preproc_stem.csv', index=False)
#data_trans_stem.to_csv('../data/preproc_trans_stem.csv', index=False)
#data_long_cmnts.to_csv('../data/preproc_long.csv', index=False)